# Regresión lineal

## Objetivo

*   Usar scikit-learn para implementar un modelo de Linear Regression
*   Crear un modelo, entrenarlo, evaluarlo y utilizarlo
*   Optimizar el modelo con regularizaciones

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import pylab as pl
import numpy as np
%matplotlib inline

In [2]:
!wget -O FuelConsumption.csv https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-ML0101EN-SkillsNetwork/labs/Module%202/data/FuelConsumptionCo2.csv

--2023-02-24 18:37:08--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-ML0101EN-SkillsNetwork/labs/Module%202/data/FuelConsumptionCo2.csv
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 169.63.118.104
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|169.63.118.104|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 72629 (71K) [text/csv]
Saving to: 'FuelConsumption.csv'

FuelConsumption.csv 100%[===================>]  70.93K  --.-KB/s    in 0.1s    

2023-02-24 18:37:09 (699 KB/s) - 'FuelConsumption.csv' saved [72629/72629]



### `FuelConsumption.csv`:

Fuel consumption dataset, **`FuelConsumption.csv`**, Contiene las calificaciones de consumo de combustible específicas del modelo y las emisiones estimadas de dióxido de carbono para vehículos ligeros nuevos para la venta al por menor en Canadá. [Dataset source](http://open.canada.ca/data/en/dataset/98f1a129-f628-4ce4-b24d-6f16bf24dd64)

*   **MODELYEAR** e.g. 2014
*   **MAKE** e.g. Acura
*   **MODEL** e.g. ILX
*   **VEHICLE CLASS** e.g. SUV
*   **ENGINE SIZE** e.g. 4.7
*   **CYLINDERS** e.g 6
*   **TRANSMISSION** e.g. A6
*   **FUELTYPE** e.g. z
*   **FUEL CONSUMPTION in CITY(L/100 km)** e.g. 9.9
*   **FUEL CONSUMPTION in HWY (L/100 km)** e.g. 8.9
*   **FUEL CONSUMPTION COMB (L/100 km)** e.g. 9.2
*   **CO2 EMISSIONS (g/km)** e.g. 182   --> low --> 0

In [9]:
df = pd.read_csv("FuelConsumption.csv")
df.sample(10)

,MODELYEAR,MAKE,MODEL,VEHICLECLASS,ENGINESIZE,CYLINDERS,TRANSMISSION,FUELTYPE,FUELCONSUMPTION_CITY,FUELCONSUMPTION_HWY,FUELCONSUMPTION_COMB,FUELCONSUMPTION_COMB_MPG,CO2EMISSIONS
22,2014,AUDI,A5 CABRIOLET QUATTRO,SUBCOMPACT,2.0,4,AS8,Z,11.5,8.1,10.0,28,230
950,2014,SUBARU,IMPREZA AWD,COMPACT,2.5,4,M6,Z,14.2,10.4,12.5,23,288
381,2014,FORD,F150 FFV,PICKUP TRUCK - STANDARD,5.0,8,AS6,X,16.4,12.3,14.6,19,336
909,2014,RAM,1500 (MDS),PICKUP TRUCK - STANDARD,5.7,8,A6,X,17.1,12.0,14.8,19,340
305,2014,DODGE,CHARGER AWD,FULL-SIZE,3.6,6,A8,X,12.8,8.6,10.9,26,251
1055,2014,VOLKSWAGEN,TOUAREG TDI CLEAN DIESEL,SUV - STANDARD,3.0,6,AS8,D,12.3,8.0,10.4,27,281
199,2014,CHEVROLET,CRUZE,MID-SIZE,1.8,4,AS6,X,10.5,6.7,8.8,32,202
143,2014,BUICK,ENCORE AWD,SUV - SMALL,1.4,4,AS6,X,10.2,8.0,9.2,31,212
1013,2014,TOYOTA,YARIS,COMPACT,1.5,4,M5,X,7.8,6.3,7.1,40,163
341,2014,FIAT,500L TURBO,STATION WAGON - SMALL,1.4,4,M6,X,9.3,7.2,8.4,34,193


In [10]:
# Vamos a utilizar solo las siguientes caracteristicas para este ejercicio
# Para X serán: 'ENGINESIZE','CYLINDERS','FUELCONSUMPTION_CITY','FUELCONSUMPTION_HWY','FUELCONSUMPTION_COMB'
# Para Y será: 'CO2EMISSIONS'
# Divide la data en train y test con un 70% para train y 30% para test

from sklearn.model_selection import train_test_split    
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

# Seleccionamos las caracteristicas que vamos a utilizar
X = df[['ENGINESIZE','CYLINDERS','FUELCONSUMPTION_CITY','FUELCONSUMPTION_HWY','FUELCONSUMPTION_COMB']]

# Seleccionamos la variable a predecir
Y = df['CO2EMISSIONS']

# Divide la data en train y test
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

# Instanciamos el modelo
regr = LinearRegression()

# Entrenamos el modelo
regr.fit(X_train, Y_train)


LinearRegression()

In [19]:
# Haz una función que entrene un modelo re regresión lineal recibiendo X_train, X_test, Y_train, Y_test
# La funcion devuelve el error cuadrático medio (MSE) y el coeficiente de determinación (R2) para train y test
# Calcula si el modelo está sobreajustado, subajustado o tiene un buen ajuste

from sklearn.metrics import mean_squared_error, r2_score

def entrenar_modelo(X_train, X_test, Y_train, Y_test):
    # Instanciamos el modelo
    regr = LinearRegression()
    
    # Entrenamos el modelo
    regr.fit(X_train, Y_train)
    
    # Hacemos las predicciones
    Y_pred_train = regr.predict(X_train)
    Y_pred_test = regr.predict(X_test)
    
    # Calculamos el MSE
    mse_train = mean_squared_error(Y_train, Y_pred_train)
    mse_test = mean_squared_error(Y_test, Y_pred_test)
    
    # Calculamos el R2
    r2_train = r2_score(Y_train, Y_pred_train)
    r2_test = r2_score(Y_test, Y_pred_test)

    # Imprime los resultados
    print("MSE train: %.2f" % mse_train)
    print("MSE test: %.2f" % mse_test)
    print("R2 train: %.2f" % r2_train)
    print("R2 test: %.2f" % r2_test)

    # calcula si el modelo está sobreajustado, subajustado o tiene un buen ajuste
    if r2_train > r2_test:  
        print("El modelo está sobreajustado")
    elif r2_train < r2_test:
        print("El modelo está subajustado")
    else:
        print("El modelo tiene un buen ajuste")
        
    return mse_train, mse_test, r2_train, r2_test

entrenar_modelo(X_train, X_test, Y_train, Y_test)

MSE train: 563.97
MSE test: 502.45
R2 train: 0.86
R2 test: 0.88
El modelo está subajustado


(563.9675185805852, 502.4460931860748, 0.8590623862972946, 0.8754316437513162)

In [43]:
# Importa ridge de sklearn.linear_model
# Crea una funcion que ajuste un modelo de regresion ridge con diferentes valores de alpha
# La funcion debe devolver el MSE y el R2 para train y test

from sklearn.linear_model import Ridge

def entrenar_modelo_ridge(X_train, X_test, Y_train, Y_test):

    # alphas
    alphas = [0.01, 0.1, 1.0, 10.0, 100.0, 1000.0]


    # Crea una tabla vacia para almacenar los resultados de Alpha, MSE y R2 para train y test
    df_ridge = pd.DataFrame(columns=['Alpha', 'MSE_train', 'MSE_test', 'R2_train', 'R2_test'])

    # Itera sobre los diferentes valores de alpha
    for alpha in alphas:
        # Instanciamos el modelo
        ridge = Ridge(alpha=alpha)
    
        # Entrenamos el modelo
        ridge.fit(X_train, Y_train)
    
        # Hacemos las predicciones
        Y_pred_train = ridge.predict(X_train)
        Y_pred_test = ridge.predict(X_test)
    
        # Calculamos el MSE
        mse_train = mean_squared_error(Y_train, Y_pred_train)
        mse_test = mean_squared_error(Y_test, Y_pred_test)
        
        # Calculamos el R2
        r2_train = r2_score(Y_train, Y_pred_train)
        r2_test = r2_score(Y_test, Y_pred_test)

        # Guarda los resultados en la tabla df_ridge con pandas.concat
        df_ridge = pd.concat([df_ridge, pd.DataFrame([[alpha, mse_train, mse_test, r2_train, r2_test]], columns=['Alpha', 'MSE_train', 'MSE_test', 'R2_train', 'R2_test'])])

    # Imprime la tabla df_ridge
    print(df_ridge)


    # ¿Que valor de alpha tiene el mejor R2, imprime todos los valores para ese alpha?
    print(df_ridge[df_ridge['R2_test'] == df_ridge['R2_test'].max()])
        
    return mse_train, mse_test, r2_train, r2_test
entrenar_modelo_ridge(X_train, X_test, Y_train, Y_test)

     Alpha   MSE_train    MSE_test  R2_train   R2_test
0     0.01  563.967776  502.439779  0.859062  0.875433
0     0.10  563.984754  502.411339  0.859058  0.875440
0     1.00  564.184921  502.503813  0.859008  0.875417
0    10.00  564.395773  502.803279  0.858955  0.875343
0   100.00  566.180055  506.472236  0.858509  0.874433
0  1000.00  600.649998  556.252220  0.849895  0.862092
   Alpha   MSE_train    MSE_test  R2_train  R2_test
0    0.1  563.984754  502.411339  0.859058  0.87544


(600.6499979535423, 556.2522202760113, 0.8498952961064707, 0.8620918230250778)

In [44]:
# Importa lasso de sklearn.linear_model
# Crea una funcion que ajuste un modelo de regresion lasso con diferentes valores de alpha
# La funcion debe devolver el MSE y el R2 para train y test


from sklearn.linear_model import Lasso

def entrenar_modelo_lasso(X_train, X_test, Y_train, Y_test):

    # alphas
    alphas = [0.1, 1.0, 10.0, 100.0, 1000.0]


    # Crea una tabla vacia para almacenar los resultados de Alpha, MSE y R2 para train y test
    df_lasso = pd.DataFrame(columns=['Alpha', 'MSE_train', 'MSE_test', 'R2_train', 'R2_test'])

    # Itera sobre los diferentes valores de alpha
    for alpha in alphas:
        # Instanciamos el modelo
        lasso = Lasso(alpha=alpha)
    
        # Entrenamos el modelo
        lasso.fit(X_train, Y_train)
    
        # Hacemos las predicciones
        Y_pred_train = lasso.predict(X_train)
        Y_pred_test = lasso.predict(X_test)
    
        # Calculamos el MSE
        mse_train = mean_squared_error(Y_train, Y_pred_train)
        mse_test = mean_squared_error(Y_test, Y_pred_test)
        
        # Calculamos el R2
        r2_train = r2_score(Y_train, Y_pred_train)
        r2_test = r2_score(Y_test, Y_pred_test)

        # Guarda los resultados en la tabla df_ridge con pandas.concat
        df_lasso = pd.concat([df_lasso, pd.DataFrame([[alpha, mse_train, mse_test, r2_train, r2_test]], columns=['Alpha', 'MSE_train', 'MSE_test', 'R2_train', 'R2_test'])])

    # Imprime la tabla df_ridge
    print(df_lasso)


    # ¿Que valor de alpha tiene el mejor R2, imprime todos los valores para ese alpha?
    print('El mejor valor de alpha es: ')
    print(df_lasso[df_lasso['R2_test'] == df_lasso['R2_test'].max()])
        
    return mse_train, mse_test, r2_train, r2_test
entrenar_modelo_lasso(X_train, X_test, Y_train, Y_test)

    Alpha    MSE_train     MSE_test  R2_train   R2_test
0     0.1   564.451348   502.717290  0.858941  0.875364
0     1.0   565.825661   504.382663  0.858598  0.874952
0    10.0   638.831620   598.389179  0.840354  0.851645
0   100.0  1378.052135  1380.533319  0.655620  0.657733
0  1000.0  4001.540141  4038.873834  0.000000 -0.001333
El mejor valor de alpha es: 
   Alpha   MSE_train   MSE_test  R2_train   R2_test
0    0.1  564.451348  502.71729  0.858941  0.875364


(4001.5401408045773, 4038.873834457047, 0.0, -0.0013330414489061138)

In [45]:
# Crea una función que normalice los datos de train y test utilizando MinMaxScaler
# Importa MinMaxScaler de sklearn.preprocessing
# Crea objeto MinMaxScaler los datos de train y test
# Ajustar el escalador a los datos de entrenamiento y transformar los datos
# Crear un objeto LinearRegression y ajustar el modelo a los datos de entrenamiento escalados
# Transformar los datos de prueba utilizando el mismo objeto escalador
# Realizar predicciones sobre los datos de prueba escalados utilizando el modelo entrenado
# Evaluar el rendimiento del modelo


from sklearn.preprocessing import MinMaxScaler

def entrenar_modelo_normalizado(X_train, X_test, Y_train, Y_test):
    
        # Instancia MinMaxScaler
        scaler = MinMaxScaler()
    
        # Ajusta el escalador a los datos de entrenamiento
        scaler.fit(X_train)
    
        # Transforma los datos de entrenamiento
        X_train_scaled = scaler.transform(X_train)
    
        # Transforma los datos de prueba
        X_test_scaled = scaler.transform(X_test)

        # Entrenamos el modelo con Least Squares
        print('Entrenando modelo con Least Squares')
        entrenar_modelo(X_train_scaled, X_test_scaled, Y_train, Y_test)

        # Entrenamos el modelo con Ridge
        print('Entrenando modelo con Ridge')
        entrenar_modelo_ridge(X_train_scaled, X_test_scaled, Y_train, Y_test)

        # Entrenamos el modelo con Lasso
        print('Entrenando modelo con Lasso')
        entrenar_modelo_lasso(X_train_scaled, X_test_scaled, Y_train, Y_test)


entrenar_modelo_normalizado(X_train, X_test, Y_train, Y_test)

Entrenando modelo con Least Squares
MSE train: 563.97
MSE test: 502.45
R2 train: 0.86
R2 test: 0.88
El modelo está subajustado
Entrenando modelo con Ridge
     Alpha    MSE_train     MSE_test  R2_train   R2_test
0     0.01   564.314803   502.632440  0.858976  0.875385
0     0.10   564.414800   503.262416  0.858951  0.875229
0     1.00   565.448529   507.058751  0.858692  0.874288
0    10.00   591.338100   543.566109  0.852222  0.865237
0   100.00  1369.259308  1365.548679  0.657817  0.661448
0  1000.00  3370.339936  3401.386804  0.157739  0.156715
   Alpha   MSE_train   MSE_test  R2_train   R2_test
0   0.01  564.314803  502.63244  0.858976  0.875385
Entrenando modelo con Lasso
    Alpha    MSE_train     MSE_test  R2_train   R2_test
0     0.1   564.847059   504.783067  0.858843  0.874852
0     1.0   606.075459   563.214881  0.848539  0.860366
0    10.0  3645.007239  3676.024516  0.089099  0.088626
0   100.0  4001.540141  4038.873834  0.000000 -0.001333
0  1000.0  4001.540141  4038.87383